# Using Hugging Face With Skills

In this notebook, we demonstrate using hugging face models for Skills using both SemanticMemory and text completions.

In [23]:
!python -m pip install ../../../python

Processing c:\users\abharris\source\repos\awharrison-28\semantic-kernel\python
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for semantic-kernel: filename=semantic_kernel-0.2.4.dev0-py3-none-any.whl size=84655 sha256=3351f87b389953dcecaa1f75564c32ed0647f6967e384af0f6dc0fd458cc504f
  Stored in directory: C:\Users\abharris\AppData\Local\Temp\pip-ephem-wheel-cache-uevh4amb\wheels\30\b7\73\59b075467a7f69e7c84459669ab7dc622b76e48d378034909c
Successfully built semantic-kernel
  Attempting uninstall: semantic-kernel
    Found existing installation: semantic-kernel 0.2.4.dev0
    Uninstalling semantic-kernel-0.2.4.dev0:
      Successfully uninstalled semantic-kernel-0.2.4.dev0



[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.hugging_face as sk_hf

SK supports Hugging Face models that can perform the following tasks: text-generation, text2text-generation, summarization, and sentence-similarity. You san search for models by task at https://huggingface.co/models. The following step may take a few minutes when run for the first time as the models will be downloaded to your local machine.

First, we will create a kernel and add both text completion and embedding services. 

For text completion, we are chosing google/flan-t5-base. This is a text2text-generation model. (Note: text-generation will repeat the input in the output, text2text-generation will not.)
For embeddings, we are using sentence-transformers/all-MiniLM-L6-v2. Vectors generated for this model are of length 384 (compared to a length of 1536 from OpenAI ADA).

In [99]:
kernel = sk.Kernel()

# Configure LLM service
kernel.config.add_text_service(
    "gpt2", sk_hf.HuggingFaceTextCompletion("gpt2", task="text-generation")
)
kernel.config.add_embedding_service(
    "sentence-transformers/all-MiniLM-L6-v2",
    sk_hf.HuggingFaceTextEmbedding("sentence-transformers/all-MiniLM-L6-v2"),
)
kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x18ef2359000>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x18ef235a110>}

### Add Memories and Define a skill to use them

Most models available on huggingface.co are not as powerful as OpenAI GPT-3+. Your skills will likely need to be simpler to accomodate this.

In [112]:
await kernel.memory.save_information_async(
    "animal-facts", id="info1", text="Sharks are fish."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info2", text="Whales are mammals."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info3", text="Penguins are birds."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info4", text="Dolphins are mammals."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info5", text="Flies are insects."
)

# Define semantic function using SK prompt template language
my_prompt = """I know these animal facts: {{recall $query1}} {{recall $query2}} {{recall $query3}} and """

# Create the semantic function
my_function = kernel.create_semantic_function(
    my_prompt, max_tokens=45, temperature=0.5, top_p=0.5
)


Let's now see what the completion looks like! Remember, "gpt2" is nowhere near as large as ChatGPT, so expect a much simpler answer.

In [113]:
context = kernel.create_new_context()
context[sk.core_skills.TextMemorySkill.COLLECTION_PARAM] = "animal-facts"
context[sk.core_skills.TextMemorySkill.RELEVANCE_PARAM] = 0.3

context["query1"] = "animal that swims"
context["query2"] = "animal that flies"
context["query3"] = "penguins are?"
output = await kernel.run_async(my_function, input_vars=context.variables)

output = str(output).strip()


query_result1 = await kernel.memory.search_async(
    "animal-facts", context["query1"], limit=1, min_relevance_score=0.3
)
query_result2 = await kernel.memory.search_async(
    "animal-facts", context["query2"], limit=1, min_relevance_score=0.3
)
query_result3 = await kernel.memory.search_async(
    "animal-facts", context["query3"], limit=1, min_relevance_score=0.3
)

print(f"gpt2 completed prompt with: '{output}'")

gpt2 completed prompt with: 'I know these animal facts: Dolphins are mammals. Flies are insects. Penguins are birds. and  I'm not even sure if they're all the same species.
But, I'm not sure if they're'
